<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desafío 1 de Procesamiento de Lenguaje Natural
## Autor: Mestanza Joaquín
## Numero de SIU: a1726

In [2]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Punto 1

In [23]:
np.random.seed(42)
#tomamos 5 documentos al azar
random_idxs = np.random.choice(len(newsgroups_train.data), 5)
print(random_idxs)

[ 7270   860  5390  5191 11284]


In [24]:
!pip install colorama

In [25]:
import colorama
from colorama import Fore

In [26]:
for i in random_idxs:
  doc = newsgroups_train.data[i]
  doc_vec = X_train[i]
  print(f"{Fore.BLUE} 📁 Document id={i} label: {newsgroups_train.target_names[y_train[i]]}")
  print(f"{Fore.WHITE}{doc[:500]}")
  cossim = cosine_similarity(doc_vec, X_train)[0]
  most_sim = np.argsort(cossim)[::-1][1:6]
  for j in most_sim:
    print(f"{Fore.RED}📝 Document id={j} similar to id={i}, with label: {newsgroups_train.target_names[y_train[j]]}")
    print(f'id={j}: {newsgroups_train.target_names[y_train[j]]}')
    related_doc = newsgroups_train.data[j]
    print(f"{Fore.WHITE}{related_doc[:500]}")
    print("\n")
  print("\n")

 📁 Document id=7270 label: rec.sport.hockey

I think Murray has done a great job.  He's picked up Ciccarelli,
Sheppard, Ysebaert, Howe, Coffey, and Riendeau (plus some depth players) 
without giving up anything the Wings needed or any of his top prospects.
All of this in three years.  Has anyone done better?

The year before he took over, the Wings didn't even make the playoffs.
There was about a year and a half during Demers' stint that the Wings
did OK, but that was due to Demers' motavational skills and clutch
and grab style.  They did
📝 Document id=1880 similar to id=7270, with label: rec.sport.hockey
id=1880: rec.sport.hockey

Devallano went earlier and more extensively to the Russian strategy
than anyone else...and was the first GM to "waste" high draft choices
on young Russians...Devallano would still be GM but he succombed to
Demers pleading to make the Oates-Federko et al trade...which is the
deal that sealed his fate.

Murray has made some decent trades...no doubt...but these

📁 **Document id=7270** label: rec.sport.hockey

### 🔗 Relacionados:
- 📝 **id=1880** label: rec.sport.hockey  
- 📝 **id=1292** label: talk.politics.mideast  
- 📝 **id=6437** label: talk.politics.mideast  
- 📝 **id=9623** label: talk.politics.mideast  
- 📝 **id=7286** label: talk.politics.mideast  

---

📁 **Document id=860** label: talk.politics.mideast  

### 🔗 Relacionados:
- 📝 **id=7714** label: comp.windows.x  
- 📝 **id=2551** label: sci.crypt  
- 📝 **id=6920** label: talk.politics.mideast  
- 📝 **id=5825** label: talk.politics.mideast  
- 📝 **id=7956** label: talk.politics.misc  

---

📁 **Document id=5390** label: alt.atheism  

### 🔗 Relacionados:
- 📝 **id=20** label: alt.atheism  
- 📝 **id=6376** label: alt.atheism  
- 📝 **id=1137** label: alt.atheism  
- 📝 **id=7491** label: alt.atheism  
- 📝 **id=10779** label: alt.atheism  

---

📁 **Document id=5191** label: rec.autos  


### 🔗 Relacionados:
- 📝 **id=8714** label: soc.religion.christian  
- 📝 **id=10779** label: alt.atheism  
- 📝 **id=6437** label: talk.politics.mideast  
- 📝 **id=5988** label: talk.politics.guns  
- 📝 **id=9623** label: talk.politics.mideast  

---

📁 **Document id=11284** label: comp.sys.mac.hardware  


### 🔗 Relacionados:
- 📝 **id=7793** label: comp.sys.mac.hardware  
- 📝 **id=4651** label: sci.electronics  
- 📝 **id=9892** label: comp.sys.mac.hardware  
- 📝 **id=3080** label: comp.sys.ibm.pc.hardware  
- 📝 **id=1079** label: rec.motorcycles  


Claramente hay categorías que no tienen nada que ver entre sí, por ejemplo en el primer documento en los relacionados se encuentra mideast, que no tiene nada en común con hockey.
Como en ese ejemplo, pasa con los otros, es decir que no lo podemos usar para asociación así como así. Para que este tipo de análisis sea correcto hay que agregar un preprocesamiento de los textos, como por ejemplo filtrar las stopwords o filtrar caracteres especiales.

### 2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [54]:
from sklearn.naive_bayes import ComplementNB
import re
# config por default:
#(input='content',
# encoding='utf-8',
# decode_error='strict',
# strip_accents=None,
# lowercase=True,
# preprocessor=None,
# tokenizer=None,
# analyzer='word',
# stop_words=None,
# token_pattern='(?u)\\b\\w\\w+\\b',
# ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.float64'>, norm='l2',
# use_idf=True, smooth_idf=True, sublinear_tf=False)

vect_configs = {
    "cfg_1": {},
    "cfg_2": { "stop_words": 'english' },
    "cfg_3": { "stop_words": 'english', "strip_accents": 'ascii'},
    "cfg_4": { "stop_words": 'english' , "max_df": 0.99, "min_df": 0.01},
    "cfg_5": { "stop_words": 'english' , "sublinear_tf": True},
    "cfg_6": { "stop_words": 'english' , "max_features": 10000},
    "cfg_7": { "stop_words": 'english' , "token_pattern": r'(?u)\b\w+\b'}, # filtro caracteres especiales
}
for cfg_name, cfg in vect_configs.items():
  print(f'{Fore.BLUE}configuracion={cfg}')
  tfidfvect = TfidfVectorizer(**cfg)
  X_train = tfidfvect.fit_transform(newsgroups_train.data)
  y_train = newsgroups_train.target
  # print(type(X_train))
  vocab = tfidfvect.get_feature_names_out()
  special_char_words = [word for word in vocab if re.search(r'[^a-zA-Z0-9]', word)]
  print(special_char_words[:10])
  # ['00_', '01_0', '023_', '02_', '02_8v', '02tm_', '03_', '05_', '05_c', '09k_']
  # print(f'Words with special characters ({len(special_char_words)} found):')
  # print(f'shape: {X_train.shape}')
  print(f'{Fore.WHITE}Cantidad de documentos: {X_train.shape[0]}')
  print(f'{Fore.WHITE}Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')
  # print(tfidfvect.vocabulary_)
  # print(f'{np.unique(newsgroups_test.target)}')
  for clf in [MultinomialNB(), ComplementNB()]:
    clf.fit(X_train, y_train)
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target
    y_pred =  clf.predict(X_test)
    print(f'F1-score {cfg_name}: {f1_score(y_test, y_pred, average="macro"):0.03f} with {clf.__class__.__name__}')

configuracion={}
['00_', '01_0', '023_', '02_', '02_8v', '02tm_', '03_', '05_', '05_c', '09k_']
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631
F1-score cfg_1: 0.585 with MultinomialNB
F1-score cfg_1: 0.693 with ComplementNB
configuracion={'stop_words': 'english'}
['00_', '01_0', '023_', '02_', '02_8v', '02tm_', '03_', '05_', '05_c', '09k_']
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101322
F1-score cfg_2: 0.647 with MultinomialNB
F1-score cfg_2: 0.694 with ComplementNB
configuracion={'stop_words': 'english', 'strip_accents': 'ascii'}
['00_', '01_0', '023_', '02_', '02_8v', '02tm_', '03_', '05_', '05_c', '09k_']
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101320
F1-score cfg_3: 0.647 with MultinomialNB
F1-score cfg_3: 0.694 with ComplementNB
configuracion={'stop_words': 'english', 'max_df': 0.99, 'min_df': 0.01}
[]
Cantidad de documentos: 11314
Tamaño 

En general
- Cambiando la configuración aumentó el f1-score (macro) de MultinomialNB.
- ComplementNB parece tener una mejor performance que MultinomiaNB y sin tener que añadir configuraciones (ver primer resultado).
- Vemos una mejora cuando realizamos un filtrado del texto. Hemos encontrado que el vocabulario estaba integrado por cosas como:

```
['00_', '01_0', '023_', '02_', '02_8v', '02tm_', '03_', '05_', '05_c', '09k_']
```
Lo cual nos indica que hay asociaciones entre letras y numeros que tampoco tienen mucho sentido y refuerzan el hecho de tener que eliminarlas ya que introducen ruido en el vocabulario.


**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [55]:
# mejor config
cfg = {'stop_words': 'english', 'token_pattern': '(?u)\\b\\w+\\b'}
tfidfvect = TfidfVectorizer(**cfg)
# esta es la matriz esparsa
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [56]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
#print(idx2word)
# elegimos
car_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'car'][0]
sports_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'sports'][0]
disk_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'disk'][0]
network_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'network'][0]
mac_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'mac'][0]

indexes = [car_idx, sports_idx, disk_idx, network_idx, mac_idx]
# print(idx2word[car_idx])

In [46]:
x_train_t = X_train.transpose()
# print(x_train_t)

In [57]:
for idx in indexes:
  print(f'Palabra elegida: {idx2word[idx]}')

  cossim = cosine_similarity(x_train_t[idx], x_train_t)[0]
  most_sim = np.argsort(cossim)[::-1][1:6]
  print('similitudes:')
  print([idx2word[el] for el in most_sim])

Palabra elegida: car
similitudes:
['cars', 'criterium', 'dealer', 'civic', 'owner']
Palabra elegida: sports
similitudes:
['wip', 'jockeys', 'rockin', 'pollute', 'admittdly']
Palabra elegida: disk
similitudes:
['hard', 'floppy', 'drive', 'boot', 'hinds']
Palabra elegida: network
similitudes:
['comit', 'beginer', 'qdeck', 'tweak', 'slowdowns']
Palabra elegida: mac
similitudes:
['se', 'tte', 'deluxe', 'macuser', 'quantums']


Si bien en cada categoria hay al menos 1 o 2 palabras relacionadas, hay algunas en las que hay palabras que no tienen relación en principio. Esto es probablemente porque el vocabulario puede mejorar añadiendo más herramientas de preprocesamiento.

Buscando en internet se encontró el siguiente [link](https://www.geeksforgeeks.org/text-preprocessing-for-nlp-tasks/#lemmatization-and-stemming) en el cual se habla de técnicas como:
- Removing stopwords (NLTK)
- Stemming and Lemmatization
- Handling Contractions
- Spell Checking

Así que intentaremos mejorar el vocabulario en base a estas técnicas.

In [59]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 6.5 MB/s eta 0:00:00


In [63]:
!pip install -U nltk
!python -c "import nltk; nltk.download('stopwords')"
!pip install symspellpy


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 10.9 MB/s eta 0:00:00


In [83]:
import nltk
import re
import contractions
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

from symspellpy import SymSpell
import pkg_resources

# Initialize SymSpell (load a dictionary)
sym_spell = SymSpell()
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
def fast_spell_check(word):
    suggestions = sym_spell.lookup(word, verbosity=2, max_edit_distance=2)
    return suggestions[0].term if suggestions else word

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

# Download required NLTK resources
# nltk.download('stopwords')
# nltk.download('wordnet')

# Initialize tools
stop_words = set(stopwords.words('english'))  # Stopwords from NLTK
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Custom text preprocessor
def custom_preprocessor(text):
    # 1. Expand contractions (e.g., "can't" → "cannot")
    text = contractions.fix(text)

    # 2. Remove special characters (keeping only words and spaces)
    # text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r"[^a-zA-Z0-9'-]+", " ", text)


    # 3. Tokenize words
    words = text.split()

    # 4. Spell check and correct each word
    # words = [fast_spell_check(word) for word in words]
      # 4. Efficient spell-checking (skip words with length < 3 to speed up)
    words = [fast_spell_check(word) if len(word) > 2 else word for word in words]


    # 5. Remove stopwords
    words = [word for word in words if word.lower() not in stop_words]

    # 6. Apply stemming and lemmatization
    # words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    # words = [lemmatizer.lemmatize(word) for word in words]
    # words = [stemmer.stem(word) for word in words]
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words]

    return ' '.join(words)

# Define TF-IDF vectorizer with custom preprocessor
tfidfvect = TfidfVectorizer(preprocessor=custom_preprocessor)

# Transform the training data
X_train = tfidfvect.fit_transform(newsgroups_train.data)


In [84]:
x_train_t = X_train.transpose()
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
car_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'car'][0]
sports_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'sport'][0]
disk_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'disk'][0]
network_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'network'][0]
mac_idx = [v for k,v in tfidfvect.vocabulary_.items() if k == 'mac'][0]

indexes = [car_idx, sports_idx, disk_idx, network_idx, mac_idx]
for idx in indexes:
  print(f'Palabra elegida: {idx2word[idx]}')

  cossim = cosine_similarity(x_train_t[idx], x_train_t)[0]
  most_sim = np.argsort(cossim)[::-1][1:6]
  print('similitudes:')
  print([idx2word[el] for el in most_sim])

Palabra elegida: car
similitudes:
['civic', 'dealer', 'mile', 'owner', 'volvo']
Palabra elegida: sport
similitudes:
['wip', 'peninsula', '3000gt', 'trainer', 'pigeonhol']
Palabra elegida: disk
similitudes:
['floppi', 'drive', 'hard', 'memorex', 'boot']
Palabra elegida: network
similitudes:
['dunedin', 'comit', 'etherrout', 'slowdown', 'localtalk']
Palabra elegida: mac
similitudes:
['se', 'delux', 'kwh', 'appl', '5396']


- Para algunas palabras se añadieron más palabras relacionadas (como 'mile' en car) y para algunas otras aparecieron otras nuevas (como 'appl' en mac).

- Se encontró una limitante de tiempo de ejecución al tratar de corregir las palabras con:
```
 words = [str(TextBlob(word).correct()) for word in words]
```
y es por eso que se reemplazo con:


```
words = [fast_spell_check(word) if len(word) > 2 else word for word in words]
```

- Hay que tener en cuenta que no hay ningún tipo de noción de contexto al obtener estas similitudes, con lo cual el resultado nos dice que las palabras pueden ser similares pero en otro sentido, no al que estamos acostumbrados.


